In [146]:
pip install faker

     |████████████████████████████████| 1.5 MB 8.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [289]:
import pandas as pd
import numpy as np
import datetime
import itertools
import random

In [368]:
Maker = "Samsung, Apple, Huawei, Google"
Pname = "iPhone 7, iPhone 8, iPhone 10, iPhone 11, iPhone 12, iPhone 13, iPhone XS, Pixel 3 API, Nexus 6 API, Samsung Galaxy S10, Samsung Galaxy S20, Samsung Galaxy S21, Huawei P30, Huawei P40, Huawei P50, iPad 11 inch, iPad 13 inch, iPad Mini"
Sname = "Samsung, Apple, Huawei, Google"
Status = "being processed, shipped, delivered, returned"
Comment = "Great product!, Slow delivery, Fast delivery"
# "MACS, Unfairprice, WorstDenki, DDDonkey, Ggiant, Pprime, Siong Sheng, KCube, Eastgate, KEM"

def str_parser(s):
    str_list = s.split(', ')
    return str_list
print(str_parser(Maker))
print(str_parser(Pname))
print(str_parser(Sname))
print(str_parser(Status))
print(str_parser(Comment))


['Samsung', 'Apple', 'Huawei', 'Google']
['iPhone 7', 'iPhone 8', 'iPhone 10', 'iPhone 11', 'iPhone 12', 'iPhone 13', 'iPhone XS', 'Pixel 3 API', 'Nexus 6 API', 'Samsung Galaxy S10', 'Samsung Galaxy S20', 'Samsung Galaxy S21', 'Huawei P30', 'Huawei P40', 'Huawei P50', 'iPad 11 inch', 'iPad 13 inch', 'iPad Mini']
['Samsung', 'Apple', 'Huawei', 'Google']
['being processed', 'shipped', 'delivered', 'returned']
['Great product!', 'Slow delivery', 'Fast delivery']



# Template


In [ ]:
def faker(rows):
    li = ['being processed', 'shipped', 'delivered', 'returned']
    result_li = [{
        'ID': x, # ID from 0
        'Worker Status': np.random.choice(li, p=[0.50, 0.30, 0.10, 0.10]) # probability
    } for x in range(rows)]
    return result_li

df = pd.DataFrame(faker(rows = 100))
df

# Shop

- Thoughts - shops are the child shops of the makers
- ~Each shop will sell products from only a maker so the product~ 
- There will be multiple shops selling products from one Maker

In [122]:
NUMBER_OF_PRODUCTS = 15 # in a shop

In [ ]:
### Shop

def faker():
    li = ['Shop 1', 'Shop 2', 'Shop 3', 'Shop 4', 'Shop 5']
    result_li = []
    for shop in li:
        for id in range(np.random.randint(5,NUMBER_OF_PRODUCTS)):
            result_li.append({'Sname':shop, 'SPID':id,'Squantity':np.random.randint(1000,3000),'SPrice':np.random.randint(500,1000)})
    return result_li

shop_df = pd.DataFrame(faker())
shop_df

# Products



In [ ]:
Maker = {
    'Apple':
        {
        'Phone': ['iPhone XS', 'iPhone 12', 'iPhone 13'],
        'Tablet': ['iPad Air', 'iPad Pro', 'iPad Mini']
        },
    'Samsung':
        {
        'Phone': ['Samsung Galaxy S10', 'Samsung Galaxy S20', 'Samsung Galaxy S21'],
        'Tablet': ['Samsung Tab A', 'Samsung Tab B', 'Samsung Tab C'],
        },
    'Huawei':
        {
        'Phone': ['Huawei P30', 'Huawei P40', 'Huawei P50'],
        'Tablet': ['Huawei Mate A', 'Huawei Mate X', 'Huawei Mate 11'],
        }
}
# 17 values

# for company in Maker: # items in Maker
#     for category in Maker[company]: # string
#         for products in Maker[company][category]:
#             print(category + ': ' + products)

def faker(rows):
    company = [company for company in Maker]
    category = [category for category in Maker['Apple']]
    li = ['Shop 1', 'Shop 2', 'Shop 3', 'Shop 4', 'Shop 5']
        
    # products = [products for products in Maker[company][category]]
    result_li = [{
        #'Sname': np.random.choice(li),
        'Maker': np.random.choice(company),
        'Category': np.random.choice(category),
        'SPID': x
    } for x in range(rows)]
    for i in range(rows):
        maker_val = result_li[i]['Maker']
        cat_val = result_li[i]['Category']
        result_li[i]['Pname'] = np.random.choice(Maker[maker_val][cat_val], p = [0.6, 0.2, 0.2])
    return result_li

product_df = pd.DataFrame(faker(rows = 50))
product_df





In [ ]:
# price = []
# for i in range(products_df.shape[0]):
#     price.append(np.random.randint(100,1000))
# products_df['Price'] = price
# products_df

### Merging products and shop

shop_product_df = shop_df.merge(product_df, how = 'left', on = 'SPID')
shop_product_df.drop_duplicates(subset = ['Sname', 'Pname'], ignore_index= True)

# around 85 possibilities, take 30 only to make sure some don't exist
# df.duplicated().sum() ### Count repeated



In [239]:
shop_product_df.to_csv("shop_product.csv", index = False)

In [264]:
shop_product_df = pd.read_csv("shop_product.csv")

In [468]:
### Adding price history to shop_product table
from faker import Faker
fake = Faker()
dfs_list = []

for index, row in shop_product_df.iterrows():
    start_date = datetime.date(year=2021, month=6, day=1)
    intermediate_date = datetime.date(year=2021, month=8, day=15)
    end_date = datetime.date(year=2021, month=8, day=31)
    date1 = pd.to_datetime(fake.date_time_between(start_date=start_date, end_date=intermediate_date))    
    date2 = pd.to_datetime(fake.date_time_between(start_date=date1, end_date=end_date))    
    old_price = shop_product_df['SPrice'].iloc[index] + np.random.randint(-100,100)
    _dict = {
        'Price': old_price,
        'Start-date': date1,
        'End-date': date2
    }
    
    # append to df list
    dfs_list.append(_dict)
    
# concatenate all the dfs 
price_df = pd.DataFrame(dfs_list)
#big_df = price_df.merge(shop_product_df, how = 'left')
price_history_merged = pd.concat([shop_product_df, price_df], axis = 1)
price_history_merged


,Sname,SPID,Squantity,SPrice,Maker,Category,Pname,Price,Start-date,End-date
0,Shop 1,0,2238,820,Huawei,Tablet,Huawei Mate 11,853,2021-07-21 12:06:16,2021-08-30 11:36:49
1,Shop 2,0,2570,853,Huawei,Tablet,Huawei Mate 11,889,2021-07-10 05:20:25,2021-07-15 11:05:22
2,Shop 3,0,1154,863,Huawei,Tablet,Huawei Mate 11,916,2021-06-27 13:38:31,2021-08-04 05:02:59
3,Shop 4,0,2302,944,Huawei,Tablet,Huawei Mate 11,997,2021-07-07 02:46:17,2021-08-27 10:15:45
4,Shop 5,0,2844,732,Huawei,Tablet,Huawei Mate 11,800,2021-08-08 23:15:05,2021-08-29 19:53:12
5,Shop 1,1,2563,961,Apple,Phone,iPhone 12,1017,2021-06-25 08:03:30,2021-07-14 03:54:26
6,Shop 2,1,1965,527,Apple,Phone,iPhone 12,591,2021-07-23 20:25:59,2021-08-10 19:41:09
7,Shop 3,1,2297,735,Apple,Phone,iPhone 12,781,2021-07-31 20:57:29,2021-08-08 10:07:26
8,Shop 4,1,2209,862,Apple,Phone,iPhone 12,860,2021-06-02 23:57:30,2021-06-17 13:17:10
9,Shop 5,1,1738,790,Apple,Phone,iPhone 12,705,2021-07-14 10:57:21,2021-07-27 18:53:11


In [282]:
price_history_merged.to_csv("shop_product_price_history.csv", index = False)

# Order

In [471]:
##Users
import random  
import string
import itertools

chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

user_name = []

n = 2
for i in range(1, n+1):
    for item in itertools.product(chars, repeat=i):
        user_name.append("".join(item))

# user_name


def make_users(num):
    
    # lists to randomly assign to worker  

    fake_users = [{'UID':x+1,
                    
                   'UName': user_name[x]
                 } for x in range(num)]
    return fake_users

users_df = pd.DataFrame(make_users(num=100))
users_df.head(30)

,UID,UName
0,1,A
1,2,B
2,3,C
3,4,D
4,5,E
5,6,F
6,7,G
7,8,H
8,9,I
9,10,J


In [480]:
chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

user_name = []

n = 2
for i in range(1, n+1):
    for item in itertools.product(chars, repeat=i):
        user_name.append("".join(item))
        
orderdf = []


lstOID = [i for i in range(1, 201)]
count1 = 200


for x in range(100):
    
    adr = np.random.randint(100000,999999) #get address
    
    randomOID = range(np.random.randint(1,3))
    li = ['Shop 1', 'Shop 2', 'Shop 3', 'Shop 4', 'Shop 5']

    for y in randomOID:
        
        index1 = np.random.randint(0,count1) #get OID
        num1 = lstOID.pop(index1)
        count1 -= 1
        
        randomOPID = range(np.random.randint(1,6))
        start_date = datetime.date(year=2021, month=6, day=1)
        end_date = datetime.date(year=2021, month=8, day=31)
        date1 = pd.to_datetime(fake.date_time_between(start_date=start_date, end_date=end_date))   

        lstOPID = [j for j in range(1,13)]
        count2 = 12
        for z in randomOPID:
            
            index2 = np.random.randint(0,count2 - 1) #get OPID
            num2 = lstOPID.pop(index2)
            count2 -= 1
            
            orderdf.append(
                {
                'UName': user_name[x - x % 2], 
                'Shipping address' : adr, 
                'OID': num1,
                'OPID' : num2,
                'Order-date-time' : date1,
                'OQuantity' : np.random.randint(1,10),
                'Status' : np.random.choice(['being processed', 'shipped', 'delivered', 'returned'], p = [0.1, 0.1, 0.7, 0.1]),
                'Sname': np.random.choice(li)
                })

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)

# Oprice, delivery-date

orderdf = pd.DataFrame(orderdf)
orderdf = orderdf.merge(users_df, how = "left", on = "UName").sort_values(by = "UID")
orderdf


,UName,Shipping address,OID,OPID,Order-date-time,OQuantity,Status,Sname,UID
0,A,167467,133,11,2021-07-20 04:16:07,5,delivered,Shop 4,1
1,A,864221,186,4,2021-06-29 09:36:36,5,being processed,Shop 5,1
2,A,864221,186,6,2021-06-29 09:36:36,1,being processed,Shop 2,1
3,A,864221,158,8,2021-08-08 10:07:43,5,returned,Shop 1,1
4,A,864221,158,6,2021-08-08 10:07:43,5,returned,Shop 2,1
...,...,...,...,...,...,...,...,...,...
375,CS,241296,163,7,2021-06-25 21:39:36,5,shipped,Shop 1,97
385,CU,164914,119,4,2021-08-28 06:56:57,6,returned,Shop 2,99
383,CU,209768,182,7,2021-06-08 21:58:47,8,shipped,Shop 5,99
384,CU,164914,119,10,2021-08-28 06:56:57,9,returned,Shop 5,99


In [481]:
orderdf.to_csv("order_UID.csv", index = False)

In [382]:
orderdf = pd.read_csv("order_UID.csv")

In [513]:
## getting the shop info to order

order_shop_merged = orderdf.merge(shop_product_df, how = "inner", left_on = ["OPID", "Sname"], right_on = ["SPID", "Sname"])
order_shop_merged['OPrice'] = order_shop_merged['SPrice']

from faker import Faker
fake = Faker()
dfs_list = []
Comment = ['Great product!', 'Slow delivery', 'Fast delivery', '']
for index, row in order_shop_merged.iterrows():
    DATE = pd.to_datetime(order_shop_merged['Order-date-time'].iloc[index])
    start_date = datetime.date(year=DATE.year, month=DATE.month, day=DATE.day)
    intermediate_date = datetime.date(year=2021, month=8, day=15)
    end_date = datetime.date(year=2021, month=8, day=31)

    date1 = pd.to_datetime(fake.date_time_between(start_date=start_date, end_date=intermediate_date)).strftime("%Y-%m-%d %H:%M:%S")
    comment = np.random.choice(Comment)
    rating = np.random.choice(range(1, 6), p = [0.1,0.1,0.1,0.1,0.6])
    date2 = pd.to_datetime(fake.date_time_between(start_date=intermediate_date, end_date=end_date)).strftime("%Y-%m-%d %H:%M:%S")  
 

    _dict = {
        'Delivery-date': date1,
        'Feedback-date-time': date2,
        'Comment': comment,
        'Rating': rating
    }
    
    # append to df list
    dfs_list.append(_dict)

# concatenate all the dfs 
tmp_df = pd.DataFrame(dfs_list)
#big_df = price_df.merge(order_shop_merged, how = 'left')
shop_order_feedback = pd.concat([order_shop_merged, tmp_df], axis = 1)
shop_order_feedback



,UName,Shipping address,OID,OPID,Order-date-time,OQuantity,Status,Sname,UID,SPID,Squantity,SPrice,Maker,Category,Pname,OPrice,Delivery-date,Feedback-date-time,Comment,Rating
0,A,167467,133,11,2021-07-20 04:16:07,5,delivered,Shop 4,1,11,1627,880,Huawei,Phone,Huawei P50,880,2021-08-08 23:10:37,2021-08-30 12:39:50,Great product!,5
1,C,755144,58,11,2021-07-22 01:38:45,9,being processed,Shop 4,3,11,1627,880,Huawei,Phone,Huawei P50,880,2021-08-08 12:27:07,2021-08-23 02:24:59,Fast delivery,5
2,E,221731,57,11,2021-07-08 07:34:34,3,being processed,Shop 4,5,11,1627,880,Huawei,Phone,Huawei P50,880,2021-08-11 05:33:16,2021-08-30 07:58:59,Great product!,1
3,AY,740841,121,11,2021-07-05 04:22:36,5,shipped,Shop 4,51,11,1627,880,Huawei,Phone,Huawei P50,880,2021-07-09 06:00:17,2021-08-22 07:15:19,,2
4,BQ,995563,26,11,2021-07-26 09:16:03,4,returned,Shop 4,69,11,1627,880,Huawei,Phone,Huawei P50,880,2021-08-01 02:38:28,2021-08-22 20:25:40,,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,CC,246560,126,4,2021-06-13 08:20:18,1,shipped,Shop 4,81,4,1177,546,Samsung,Phone,Samsung Galaxy S10,546,2021-07-19 21:59:40,2021-08-17 11:11:00,Great product!,2
344,CG,200595,151,4,2021-07-16 21:33:59,8,shipped,Shop 4,85,4,1177,546,Samsung,Phone,Samsung Galaxy S10,546,2021-07-22 06:48:18,2021-08-22 03:35:30,Great product!,5
345,BQ,995563,26,8,2021-07-26 09:16:03,5,returned,Shop 3,69,8,2888,662,Samsung,Phone,Samsung Galaxy S20,662,2021-08-07 05:07:55,2021-08-22 15:51:30,,5
346,BW,651787,165,8,2021-08-03 04:28:46,9,delivered,Shop 3,75,8,2888,662,Samsung,Phone,Samsung Galaxy S20,662,2021-08-14 11:50:44,2021-08-20 10:19:54,,3


In [514]:
shop_order_feedback.to_csv("shop_order_feedback.csv", index = False)
df4 = pd.read_csv('shop_order_feedback.csv')
Feedback = df4[["Pname", "UID", "Feedback-date-time", "Rating", "Comment"]]
Feedback = Feedback.drop_duplicates()
Feedback.to_csv('Feedback_v3.csv', index = False)


In [453]:
shop_order_feedback = pd.read_csv("shop_order_feedback.csv")

# Complaints on shop, complains on product

In [484]:
### Generate Employees

chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

employees_name = []

n = 2
for i in range(1, n+1):
    for item in itertools.product(chars, repeat=i):
        employees_name.append("".join(item))

def gen_employee(rows):
    result_empl = [{
        'EID': x+1,
        'Employee name': employees_name[x],
        'Salary': np.random.randint(20,100) *100
    } for x in range(rows)]
    return result_empl

dfemp = pd.DataFrame(gen_employee(rows = 50))
dfemp

,EID,Employee name,Salary
0,1,A,5900
1,2,B,7700
2,3,C,8000
3,4,D,6100
4,5,E,3100
5,6,F,4700
6,7,G,2500
7,8,H,5300
8,9,I,5800
9,10,J,8000


In [485]:
dfemp.to_csv("employee_only.csv", index = False)


In [404]:
employee_only = pd.read_csv("employee_only.csv")

In [505]:
# Complaints on shop

def make_complaints(num):
    # lists to randomly assign to workers
    status_list = ['pending', 'being handled', 'addressed']  
    text_list = ['Not good', 'Bad vibes', 'Not worth', 'False advertising', 'Bad service']  
    li = ['Shop 1', 'Shop 2', 'Shop 3', 'Shop 4', 'Shop 5']

    fake_complaints =[{'CID':x,
                        'Complain-status':np.random.choice(status_list, p=[0.40, 0.30, 0.30]), 
                        'Complain-text':np.random.choice(text_list),
                        'EID': random.randint(1,100),
                        'OID': random.randint(1,200),
                        'Sname': np.random.choice(li)
                    } for x in range(num)]
    return fake_complaints

complaints_shop_df = pd.DataFrame(make_complaints(num=50))
complaints_shop_df.head(30)

,CID,Complain-status,Complain-text,EID,OID,Sname
0,0,addressed,Not good,59,127,Shop 1
1,1,being handled,Not good,15,167,Shop 2
2,2,addressed,Not good,87,152,Shop 5
3,3,being handled,Not worth,14,117,Shop 1
4,4,addressed,Not good,17,81,Shop 1
5,5,being handled,Bad vibes,47,123,Shop 4
6,6,addressed,Bad vibes,100,82,Shop 1
7,7,pending,False advertising,96,48,Shop 5
8,8,addressed,False advertising,50,135,Shop 1
9,9,pending,False advertising,86,120,Shop 1


In [506]:
complaint_shops_merged = complaints_shop_df.merge(dfemp, how = "inner", on = "EID")
complaint_shops_merged = shop_order_feedback.merge(complaint_shops_merged, how = "inner", on = ["OID", "Sname"])
complaint_shops_merged

from faker import Faker
fake = Faker()
dfs_list = []
for index, row in complaint_shops_merged.iterrows():
    DATE = pd.to_datetime(complaint_shops_merged['Order-date-time'].iloc[index])
    start_date = datetime.date(year=DATE.year, month=DATE.month, day=min(DATE.day + 1, 30))
    intermediate_date = datetime.date(year=2021, month=8, day=DATE.day)
    end_date = datetime.date(year=2021, month=8, day=31)

    date1 = pd.to_datetime(fake.date_time_between(start_date=start_date, end_date=intermediate_date)).strftime("%Y-%m-%d %H:%M:%S")
    date2 = pd.to_datetime(fake.date_time_between(start_date=intermediate_date, end_date=end_date)).strftime("%Y-%m-%d %H:%M:%S")  
    _dict = {
        'Filed-date-time': date1,
        'Handled-date-time': date2,
    }
    
    # append to df list
    dfs_list.append(_dict)

# concatenate all the dfs 
tmp_df = pd.DataFrame(dfs_list)
#big_df = price_df.merge(complaint_shops_merged, how = 'left')
complaints_shops_date = pd.concat([complaint_shops_merged, tmp_df], axis = 1)
complaints_shops_date = complaints_shops_date[complaints_shops_date['Handled-date-time']>complaints_shops_date['Filed-date-time']]
complaints_shops_date



,UName,Shipping address,OID,OPID,Order-date-time,OQuantity,Status,Sname,UID,SPID,...,Comment,Rating,CID,Complain-status,Complain-text,EID,Employee name,Salary,Filed-date-time,Handled-date-time
0,BO,393926,152,8,2021-06-16 10:19:00,7,returned,Shop 1,67,8,...,Slow delivery,5,44,pending,Bad vibes,20,T,7500,2021-08-12 04:41:01,2021-08-21 01:18:45
1,C,755144,58,4,2021-07-22 01:38:45,6,delivered,Shop 3,3,4,...,,5,20,being handled,Not worth,39,AM,5000,2021-08-20 09:47:26,2021-08-30 09:57:58
2,S,849952,50,6,2021-08-18 18:01:13,2,being processed,Shop 3,19,6,...,Great product!,0,27,addressed,Bad service,6,F,4700,2021-08-19 00:00:00,2021-08-23 02:48:41
3,CK,944356,23,2,2021-08-14 04:55:42,1,being processed,Shop 2,89,2,...,Fast delivery,3,40,being handled,Bad vibes,14,N,8200,2021-08-15 00:00:00,2021-08-21 17:53:46
4,CK,944356,23,3,2021-08-14 04:55:42,2,shipped,Shop 2,89,3,...,Great product!,5,40,being handled,Bad vibes,14,N,8200,2021-08-15 00:00:00,2021-08-19 09:04:57
5,BA,145982,167,10,2021-06-27 12:14:16,8,delivered,Shop 2,53,10,...,Great product!,5,1,being handled,Not good,15,O,9800,2021-07-08 09:03:36,2021-08-29 17:57:53
6,AA,506408,135,7,2021-06-13 20:55:46,8,returned,Shop 1,27,7,...,,5,8,addressed,False advertising,50,AX,7800,2021-07-21 03:06:02,2021-08-19 21:43:44
7,BU,116718,160,4,2021-06-03 12:01:58,4,returned,Shop 2,73,4,...,Great product!,5,39,addressed,Bad service,16,P,9900,2021-07-06 21:33:21,2021-08-18 20:49:12


In [507]:
complaints_shops_date.to_csv("complaints_shops_date.csv", index = False)


In [508]:
# Complaints on products

def make_complaints(num):
    # lists to randomly assign to workers
    status_list = ['pending', 'being handled', 'addressed']  
    text_list = ['Bad vibes', 'Faulty on arrival', 'Late delivery', 'Damaged packaging', 'Regretted buying', 'Not worth', ]  

    fake_complaints =[{'CID':x + 50,
                        'Complain-status':np.random.choice(status_list, p=[0.40, 0.30, 0.30]), 
                        'Complain-text':np.random.choice(text_list),
                        'EID': random.randint(1,100),
                        'OID': np.random.randint(0,13)
                    } for x in range(num)]
    return fake_complaints

complaints_products_df = pd.DataFrame(make_complaints(num=50))
complaints_products_df.head(30)

,CID,Complain-status,Complain-text,EID,OID
0,50,being handled,Faulty on arrival,86,7
1,51,being handled,Late delivery,56,2
2,52,pending,Damaged packaging,94,9
3,53,pending,Late delivery,82,0
4,54,being handled,Damaged packaging,37,6
5,55,being handled,Bad vibes,38,7
6,56,being handled,Regretted buying,31,1
7,57,being handled,Damaged packaging,38,2
8,58,pending,Bad vibes,91,2
9,59,addressed,Late delivery,12,0


In [509]:
complaint_products_merged = complaints_products_df.merge(dfemp, how = "inner", on = "EID")
complaint_products_merged

,CID,Complain-status,Complain-text,EID,OID,Employee name,Salary
0,54,being handled,Damaged packaging,37,6,AK,2900
1,71,being handled,Faulty on arrival,37,4,AK,2900
2,55,being handled,Bad vibes,38,7,AL,4300
3,57,being handled,Damaged packaging,38,2,AL,4300
4,69,pending,Bad vibes,38,0,AL,4300
5,56,being handled,Regretted buying,31,1,AE,8900
6,59,addressed,Late delivery,12,0,L,9500
7,74,pending,Damaged packaging,12,9,L,9500
8,62,pending,Late delivery,8,9,H,5300
9,63,addressed,Regretted buying,43,8,AQ,4200


In [510]:
# Merging complaints and add filed date

complaints_products_merged = shop_order_feedback.merge(complaints_products_df, how = "inner", on = "OID")

from faker import Faker
fake = Faker()
dfs_list = []
for index, row in complaints_products_merged.iterrows():
    DATE = pd.to_datetime(complaints_products_merged['Order-date-time'].iloc[index])
    start_date = datetime.date(year=DATE.year, month=DATE.month, day=min(DATE.day + 1, 30))
    intermediate_date = datetime.date(year=2021, month=start_date.month, day=DATE.day)
    
    end_date = datetime.date(year=2021, month=8, day=31)

    date1 = pd.to_datetime(fake.date_time_between(start_date=start_date, end_date=intermediate_date)).strftime("%Y-%m-%d %H:%M:%S")
    date2 = pd.to_datetime(fake.date_time_between(start_date=intermediate_date, end_date=end_date)).strftime("%Y-%m-%d %H:%M:%S")  
    _dict = {
        'Filed-date-time': date1,
        'Handled-date-time': date2,
    }
    
    # append to df list
    dfs_list.append(_dict)

# concatenate all the dfs 
tmp_df = pd.DataFrame(dfs_list)
#big_df = price_df.merge(complaints_products_merged, how = 'left')
complaints_products_date = pd.concat([complaints_products_merged, tmp_df], axis = 1)
complaints_products_date = complaints_products_date[complaints_products_date['Filed-date-time']<complaints_products_date['Handled-date-time']]
complaints_products_date



,UName,Shipping address,OID,OPID,Order-date-time,OQuantity,Status,Sname,UID,SPID,...,Delivery-date,Feedback-date-time,Comment,Rating,CID,Complain-status,Complain-text,EID,Filed-date-time,Handled-date-time
0,BQ,422925,11,7,2021-08-02 11:31:14,2,shipped,Shop 3,69,7,...,NaN,NaN,,4,70,pending,Regretted buying,52,2021-08-03 00:00:00,2021-08-23 02:25:38
1,BQ,422925,11,7,2021-08-02 11:31:14,2,shipped,Shop 3,69,7,...,NaN,NaN,,4,86,addressed,Damaged packaging,17,2021-08-03 00:00:00,2021-08-08 23:19:31
2,BQ,422925,11,7,2021-08-02 11:31:14,2,shipped,Shop 3,69,7,...,NaN,NaN,,4,93,being handled,Not worth,83,2021-08-03 00:00:00,2021-08-16 05:36:25
3,BQ,422925,11,6,2021-08-02 11:31:14,6,delivered,Shop 3,69,6,...,2021-08-07 04:18:03,2021-08-23 07:32:21,Slow delivery,5,70,pending,Regretted buying,52,2021-08-03 00:00:00,2021-08-30 18:54:01
4,BQ,422925,11,6,2021-08-02 11:31:14,6,delivered,Shop 3,69,6,...,2021-08-07 04:18:03,2021-08-23 07:32:21,Slow delivery,5,86,addressed,Damaged packaging,17,2021-08-03 00:00:00,2021-08-06 20:29:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,CA,524924,4,4,2021-06-08 19:35:50,3,returned,Shop 4,79,4,...,NaN,NaN,Fast delivery,5,82,addressed,Faulty on arrival,76,2021-06-09 00:00:00,2021-07-20 20:12:58
94,CA,524924,4,4,2021-06-08 19:35:50,3,returned,Shop 4,79,4,...,NaN,NaN,Fast delivery,5,84,being handled,Late delivery,22,2021-06-09 00:00:00,2021-07-15 21:24:39
95,CA,524924,4,4,2021-06-08 19:35:50,3,returned,Shop 4,79,4,...,NaN,NaN,Fast delivery,5,91,pending,Bad vibes,82,2021-06-09 00:00:00,2021-08-07 09:31:59
96,CA,524924,4,4,2021-06-08 19:35:50,3,returned,Shop 4,79,4,...,NaN,NaN,Fast delivery,5,96,being handled,Bad vibes,91,2021-06-09 00:00:00,2021-06-29 07:50:30


In [511]:
complaints_products_date.to_csv("complaints_products_date.csv", index = False)

Need to merge from complaint shop, complaint product into products

In [421]:
all_complaints_merged[all_complaints_merged['CID'].notnull()] # show not null


,UName,Shipping address,OID,OPID,Order-date-time,OQuantity,Status,Sname,UID,SPID,...,OPrice,Price,Delivery-date,Feedback-date-time,Comment,Rating,CID,Complain-status,Complain-text,EID
1,B,588701,4,4,2021-06-07 12:47:31,9,returned,Shop 5,2,4,...,969,605,NaN,NaN,NaN,5,59.0,pending,Faulty on arrival,74.0
2,B,588701,4,4,2021-06-07 12:47:31,9,returned,Shop 5,2,4,...,969,605,NaN,NaN,NaN,5,64.0,being handled,Damaged packaging,95.0
3,B,588701,4,4,2021-06-07 12:47:31,9,returned,Shop 5,2,4,...,969,605,NaN,NaN,NaN,5,76.0,pending,Faulty on arrival,48.0
4,B,588701,4,4,2021-06-07 12:47:31,9,returned,Shop 5,2,4,...,969,605,NaN,NaN,NaN,5,85.0,addressed,Damaged packaging,93.0
5,B,588701,4,4,2021-06-07 12:47:31,9,returned,Shop 5,2,4,...,969,605,NaN,NaN,NaN,5,88.0,being handled,Not worth,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,AF,185034,2,6,2021-07-24 14:11:07,2,being processed,Shop 3,32,6,...,930,605,NaN,NaN,NaN,5,91.0,pending,Faulty on arrival,26.0
495,AF,185034,2,9,2021-07-24 14:11:07,4,being processed,Shop 5,32,9,...,723,605,NaN,NaN,NaN,5,69.0,pending,Regretted buying,55.0
496,AF,185034,2,9,2021-07-24 14:11:07,4,being processed,Shop 5,32,9,...,723,605,NaN,NaN,NaN,5,81.0,pending,Not worth,88.0
497,AF,185034,2,9,2021-07-24 14:11:07,4,being processed,Shop 5,32,9,...,723,605,NaN,NaN,NaN,5,87.0,pending,Damaged packaging,88.0


In [ ]:
order_complaint_merged = shop_order_feedback.merge(complaints_df, how = "inner", on = "UID")


## Online stuff

In [29]:
def make_workers(num):
    # lists to randomly assign to workers
    status_list = ['Full Time', 'Part Time', 'Per Diem']    

    fake_workers = [{'Worker ID':x+1000,
                    
                    'Worker Status':np.random.choice(status_list, p=[0.50, 0.30, 0.20]), # assign items from list with different probabilities
                    } for x in range(num)]
    return fake_workers

worker_df = pd.DataFrame(make_workers(num=5000))
worker_df.head()

,Worker ID,Worker Status
0,1000,Per Diem
1,1001,Part Time
2,1002,Full Time
3,1003,Per Diem
4,1004,Per Diem


In [183]:
# function to create widget data

def make_widget_data(num):
    
    fake_widgets = [{'Item Number':id(y),
                     'Step 1':np.random.gamma(shape=3, scale=1),
                     'Step 2':np.random.normal(5), 
                     'Step 3':np.random.exponential(4)} for y in range(num)]
    
    return fake_widgets


# empty list to store our widget dataframes in    
dfs_list = []

# now lets make some widget data for each worker
# iterate through the worker dataframe
for index, row in worker_df.iterrows():
    
    # not all workers work at the same rate - or the same number of hours
    # randomly select a number of widgets for them to create based on 'worker status'
    if row['Worker Status'] == 'Full Time':
        num_widgets = np.random.randint(500, 1000)
    elif row['Worker Status'] == 'Part Time':
        num_widgets = np.random.randint(100, 500)
    else:
        num_widgets = np.random.randint(1, 1000)
    
    # make widgets for each worker
    tmp_widgets = pd.DataFrame(make_widget_data(num=num_widgets))
    
    # add worker id so we know who made the widget
    tmp_widgets['Worker ID'] = row['Worker ID']
    
    # make sure item number is unique by appending worker id
    tmp_widgets['Item Number'] = tmp_widgets['Item Number'].astype('str')+ '-' + tmp_widgets['Worker ID'].astype('str')
    
    # append to df list
    dfs_list.append(tmp_widgets)
    
# concatenate all the dfs 
widget_df = pd.concat(dfs_list)
print(widget_df.shape)
widget_df.head()

KeyboardInterrupt: 

In [ ]:
pip install faker

from faker import Faker
fake = Faker()

fake.date_between(start_date='today', end_date='+30y')
# datetime.date(2025, 3, 12)

fake.date_time_between(start_date='-30y', end_date='now')
# datetime.datetime(2007, 2, 28, 11, 28, 16)

# Or if you need a more specific date boundaries, provide the start 
# and end dates explicitly.
import datetime
start_date = datetime.date(year=2021, month=6, day=1)
fake.date_between(start_date=start_date, end_date='+3m')